In [122]:
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt


In [123]:
x_0 = torch.linspace(-2, 2, 500)
x_1 = torch.linspace(-1, 1, 500)
y_0 = x_1 + x_0
y_1 = x_1 - x_0
x_0, x_1 = torch.unsqueeze(x_0, dim=1), torch.unsqueeze(x_1, dim=1)
y_0, y_1 = torch.unsqueeze(y_0, dim=1), torch.unsqueeze(y_1, dim=1)


In [124]:
SCALE = 2

def y_0_clean(x_0, x_1):
    return x_1 + x_0
def y_0_target(x_0,x_1):
    return y_0_clean(x_0,x_1) + (SCALE*torch.rand(x_0.size()) - 0.5*SCALE)

def y_1_clean(x_0, x_1):
    return x_1 - x_0
def y_1_target(x_0, x_1 ):
    return y_1_clean(x_0,x_1) + (SCALE*torch.rand(x_0.size()) - 0.5*SCALE)


In [125]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=2, out_features=1000),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=1000, out_features=2),
)


In [126]:
mse_loss = nn.MSELoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

print(x_0.size(), x_1.size(), y_0.size(), y_1.size())

x_train = torch.cat((x_0, x_1), dim=1)

y_train = torch.cat((y_0_target(x_0,x_1), y_1_target(x_0, x_1)), dim=1)

print(x_train.size())

for step in range(2000):
    pre_y = model(x_train)
    mse_y = mse_loss(pre_y, y_train)
    kl = kl_loss(model)
    cost = mse_y+ kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
print('- MSE : %2.2f ' % (mse_y.item()))


torch.Size([500, 1]) torch.Size([500, 1]) torch.Size([500, 1]) torch.Size([500, 1])
torch.Size([500, 2])


- MSE : 0.37 


In [129]:
x_0_test = torch.linspace(-2, 2, 500)
x_1_test = torch.linspace(-1, 1, 500)

x_0_test, x_1_test = torch.unsqueeze(x_0_test, dim=1), torch.unsqueeze(x_1_test, dim=1)

y_test = torch.cat((y_0_target(x_0_test,x_1_test), y_1_target(x_0_test, x_1_test)), dim=1)

x_test = torch.cat((x_0_test, x_1_test), dim=1)

print(x_test.size(), y_test.size())


models_result = np.array([model(x_test).data.numpy() for k in range(10000)])
#print(models_result.shape)
#models_result = models_result[:,:,0]
#print(models_result.shape)   
#models_result = models_result.T
#print(models_result.shape)  
#mean_values = np.array([models_result[i].mean() for i in range(len(models_result))])
#std_values = np.array([models_result[i].std() for i in range(len(models_result))])

#print(mean_values)
# Transpose the result to have dimensions [num_predictions, 500, 2]
models_result = models_result.transpose(0, 2, 1)

# Calculate mean and std separately for each column
mean_values = models_result.mean(axis=0)
std_values = models_result.std(axis=0)

# Transpose back to have dimensions [500, 2]
mean_values = mean_values.transpose(1, 0)
std_values = std_values.transpose(1, 0)

#print(mean_values)
#print(y_test)


torch.Size([500, 2]) torch.Size([500, 2])


In [128]:
mean_values_tensor = torch.tensor(mean_values, dtype=torch.float32)
test_mse = mse_loss(mean_values_tensor, y_test)
print(f'MSE: {test_mse.item():2.4f}')


MSE: 0.3347
